In [1]:
import numpy as np
import cv2 as cv

def draw_line_segments(line_segments,img):
    img2 = img.copy()
    #img2 = cv.cvtColor(img2, cv.COLOR_GRAY2RGB)
    if line_segments is not None: # Könnte None sein, wenn Hough keine Linie erkennt!
        for line in line_segments:
            x1,y1,x2,y2 = line[0]
            cv.line(img2,(x1,y1),(x2,y2),(255,0,0),2)
    return img2

def hsv_helper(img):
    def nothing(x):
        pass
    # Create a window
    window_id='Press q to quit'
    cv.namedWindow(window_id)
    # create trackbars for color change
    cv.createTrackbar('HMin',window_id,0,255,nothing) # Hue is from 0-179 for Opencv
    cv.createTrackbar('SMin',window_id,0,255,nothing)
    cv.createTrackbar('VMin',window_id,0,255,nothing)
    cv.createTrackbar('HMax',window_id,0,255,nothing)
    cv.createTrackbar('SMax',window_id,0,255,nothing)
    cv.createTrackbar('VMax',window_id,0,255,nothing)
    # Set default value for MAX HSV trackbars.
    cv.setTrackbarPos('HMax', window_id, 255)
    cv.setTrackbarPos('SMax', window_id, 255)
    cv.setTrackbarPos('VMax', window_id, 255)
    # Initialize to check if HSV min/max value changes
    global hMin
    global sMin
    global vMin
    global hMax
    global sMax
    global vMax
    global upper 
    global lower

    output = img
    while(1):
        # get current positions of all trackbars 
        hMin = cv.getTrackbarPos('HMin',window_id)
        sMin = cv.getTrackbarPos('SMin',window_id)
        vMin = cv.getTrackbarPos('VMin',window_id)
        hMax = cv.getTrackbarPos('HMax',window_id)
        sMax = cv.getTrackbarPos('SMax',window_id)
        vMax = cv.getTrackbarPos('VMax',window_id)
        # Set minimum and max HSV values to display
        lower = np.array([hMin, sMin, vMin])
        upper = np.array([hMax, sMax, vMax])
        # Create HSV Image and threshold into a range.
        hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask = cv.inRange(hsv, lower, upper)
        output = cv.bitwise_and(img,img, mask= mask)
        cv.imshow(window_id,output)
        if cv.waitKey(30) == ord('q'):
            break
    cv.destroyAllWindows()

# Erstellen eines Objektes für den Kamerazugriff
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

# Parameter canny
maxVal=150
minVal=50
# Parameter Hough
rho = 1              # distance precision in pixel, i.e. 1 pixel
angle = np.pi / 180  # angular precision in radian, i.e. 1 degree
min_threshold = 50   # in etwas Anzal der Punkt auf der Geraden. Je geringer Min_threshold, dest mehr Geraden werden erkannt.
minLineLength = 30   # Minimale Linienlänge
maxLineGap = 10       # Maximale Anzahl von Lücken in der Linie
# Parameter Test
#f = cv.FONT_HERSHEY_SIMPLEX
#c = (0,255,255)

#### MAIN PROGRAM ####

answer = input('Which filter for edge detection?\n a) Gray\n b) HSV')

# Schleife für Video Capturing
while True:
    # Abfrage eines Frames
    ret, frame = cap.read()
    height, width, _ = frame.shape
    frame = cv.resize(frame,(int(width*1/3), int(height*1/3)), interpolation = cv.INTER_CUBIC)
    frame = cv.flip(frame,0)
    frame = cv.flip(frame,1)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    # Wenn ret == TRUE, so war Abfrage erfolgreich
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Bildmanipulation ----------
    #GRAY
    if answer =='a':
        output = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    #HSV
    if answer =='z':
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        mask = cv.inRange(hsv, lower, upper)
        output = cv.bitwise_and(frame,frame, mask= mask)
    #HSV erster Durchlauf der while Schleife mit Einstell - Tool
    if answer == 'b':
        hsv_helper(frame)
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        mask = cv.inRange(hsv, lower, upper)
        output = cv.bitwise_and(frame,frame, mask= mask)
        answer = 'z'
    #frame_gray_blurred=cv.blur(frame_gray,(5,5))
    frame_canny = cv.Canny(output,maxVal,maxVal)
    line_segments = cv.HoughLinesP(frame_canny, rho, angle, min_threshold, np.array([]), minLineLength=minLineLength, maxLineGap=maxLineGap)
    frame_ls = draw_line_segments(line_segments,frame)
    #frame_to_display = np.hstack((frame,frame_ls)) # Numpy-Operationen sind auch möglich!

    
    #frame_to_display_additional = np.hstack((frame_gray_blurred,frame_canny))
    #frame_to_display_additional = cv.cvtColor(frame_to_display_additional, cv.COLOR_GRAY2BGR)

    #frame_to_display_final = np.vstack((frame_to_display,frame_to_display_additional))
    #cv.putText(frame_to_display_final, text = 'Original', org=(10,303), fontFace= f,fontScale = .5, color = c, thickness = 1)
    #cv.putText(frame_to_display_final, text = 'Graustufen mit Smoothing', org=(10,616), fontFace= f,fontScale = .5, color = c, thickness = 1)
    #cv.putText(frame_to_display_final, text = 'Hough-Liniensegmente', org=(445,303), fontFace= f,fontScale = .5, color = c, thickness = 1)
    #cv.putText(frame_to_display_final, text = 'Canny-Kantenerkennung', org=(445,616), fontFace= f,fontScale = .5, color = c, thickness = 1)
    
    # ---------------------------
    # Alternative:
    # frame_to_display = cv.cvtColor(frame_ls, cv.COLOR_BGR2GRAY)
    # ---------------------------
    # Display des Frames
    cv.imshow("Display window (press q to quit)", frame_ls)
    # Ende bei Drücken der Taste q
    if cv.waitKey(1) == ord('q'):
        break
# Kamera-Objekt muss "released" werden, um "später" ein neues Kamera-Objekt erstellen zu können!!!
cap.release()
cv.destroyAllWindows()

[ WARN:0@3.835] global /home/pi/opencv/modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
Unable to init server: Verbindung ist gescheitert: Verbindungsaufbau abgelehnt


error: OpenCV(4.5.5) /home/pi/opencv/modules/highgui/src/window_gtk.cpp:635: error: (-2:Unspecified error) Can't initialize GTK backend in function 'cvInitSystem'
